# Import modules

In [1]:
# -*- coding: utf-8 -*-
import numpy
import random
from matplotlib import pyplot as plt
from math import exp
%matplotlib inline

# Initial parameters

In [2]:
# Population size
n = 2

# Default Time beginning
t=0

# Default trial duration
duration = 2.00 #second

# Default Time resolution
dt = 0.0001 #second 0.0001
 
# Thresholds (if from paper x200) 
Cortex_h   = 20.0             
Striatum_h = 2.0                 #depends on dopamine level, see below
Stn_h      = -17.0              
Gpi_h      = 20.0               
Thalamus_h = -47.0              

####################################### Weights ##########################################
CtxTha_G = 0.97                               
StnCtx_G = 2.0                             
StrCtx_G = 0.45                               #depends on dopamine level, see below
GpiStr_G = 12.0                              
GpiStn_G = 3.4                              
ThaGpi_G = 0.3        

# Time constants 
CtxTha_tau = 0.005 #second
StnCtx_tau = 0.020 #second "5ms for all of the synapses except synapses from cortex to STN for which it's 20ms"
StrCtx_tau = 0.005 #second
GpiStr_tau = 0.005 #second
GpiStn_tau = 0.005 #second
ThaGpi_tau = 0.005 #second


# inputs: of each neurons of populations
Cortex_I   = []
Striatum_I = [] 
Stn_I      = [] 
Gpi_I      = [] 
Gpi_Ie     = [] 
Gpi_Ii     = [] 
Thalamus_I = [] 

# Activities at time t: of each neurons of populations
CtxTha_m = []
StnCtx_m = [] 
StrCtx_m = [] 
GpiStr_m = [] 
GpiStn_m = []
ThaGpi_m = [] 

# Activities at dt: of each neurons of populations
CtxTha_dm  = [] 
StrCtx_dm  = [] 
StnCtx_dm  = [] 
GpiStr_dm  = [] 
GpiStn_dm  = [] 
ThaGpi_dm  = [] 

# Delays:
CtxTha_D = 0.005
StnCtx_D = 0.005
StrCtx_D = 0.006
GpiStr_D = 0.01
GpiStn_D = 0.005
ThaGpi_D = 0.005

# Initialization of the random generator (reproductibility)
numpy.random.seed(1)

# Dopamine dependency : Weight & Threshold

In [3]:
# relative level of Striatal Dopamine Vs Normal (given in %)
#
#def StrCtx_DG(D):                          #function calculating the Cortex_Striatum Weight 
#    """The function give the weight of the synapse between the Ctx and Str depending on the dopamine level
#    according to the 2006 article(A.Leblois et al)"""
#    
#    StrCtx_G = 0.75/(1+exp(-0.09*(D-60)))  #depending on the relative Dopamine level D (given in %)
#    return StrCtx_G
#
#def Striatum_Dh(D):                                                  #function calculating the Striatum 
#    """The function give the threshold of the striatum depending on the dopamine level
#    according to the 2006 article(A.Leblois et al)"""
#    
#    Striatum_h = -0.02 + 0.03 * (1-(1.1/(1+0.1*exp(-0.03*(D-100))))) #Treshold depending on the relative 
#    return Striatum_h                                                #Dopamine level D (given in %)

# Noise Function

In [4]:
#Noise level (%)  = sigma from paper
Cortex_N   =   0.030 
Striatum_N =   0.005
Stn_N      =   0.02
Gpi_N      =   0.050
Thalamus_N =   0.05

In [5]:
def noise(Z, level):    #Z = size of the population
        """generates Z random numbers between a chosen minimum and maximum;we could add a noise level 
    depending on the structure we are looking (e.g, more noise in the cortex than in the striatum)  """
        Z = (numpy.random.uniform(-level/2,level/2,Z))*Z
        return Z

# Basal Ganglia Population : Connectivity

In [6]:
# We set up at the beggining of the program which neuron is connected to which 
# and we keep them in the appropriate list until the end

CtxTha_Ji=[]
StrCtx_Ji=[]
StnCtx_Ji=[]
GpiStn_Ji=[]
GpiStr_Ji=[]
ThaGpi_Ji=[]

def connectivity_J(population_J,Co):
    """Return a 1-D array full of 1 or 0 depending of the probability chosen. Co is the probability 
    to find a connexion : 1=100%"""
    
    upgrade_j=0                             # va permettre de compter chaque iteration(jusqu'a n)
    while upgrade_j <= n:                   # tant que j n'as pas atteint n (= le nb de neurone dans une populat°)
        population_Ji = numpy.random.choice(2,n, p=[(1-Co),Co]) #on genere une liste de n chiffre de 1 et 0
        population_J.append(population_Ji)      #liste correspond a la connectivité du neurone i sur le neurone j
        upgrade_j += 1                      # on ajoute cette connectivity a une liste, puis on recommence avec le
                                            # j suivant jusqu'a n pour avoir toute les connectivité au préalable
    a = numpy.reshape(population_J,(n+1,n))  # on reshape notre liste pour pouvoir les transposer ensuite
    return a.transpose()                       # !!!! rend les collones en lignes et les lignes collones
                                               # permet de mettre dans une meme liste l'activité du neurone 1(ligne)
                                               # dans une meme liste(j=collones): rendra plus simple le calcul des
                                               # inputs totaux
            
# ici on devrais donc avoir un array avec sur les lignes les connectivité du neurone [i] 
# avec les (collones) neurones j. pour calculer la somme des inputs(Jij), il suffira de prendre 
# la liste du neurone [i] qui nous interesse et faire les operation dessus

In [7]:
# Connectivity J
CtxTha_J = connectivity_J(CtxTha_Ji,0.500)
StrCtx_J = connectivity_J(StrCtx_Ji,0.909)
StnCtx_J = connectivity_J(StnCtx_Ji,0.092)
GpiStn_J = connectivity_J(GpiStn_Ji,0.446)
GpiStr_J = connectivity_J(GpiStr_Ji,0.048)
ThaGpi_J = connectivity_J(ThaGpi_Ji,0.333)

# Corrected input function

In [8]:
def Ic(Input, threshold):           # Input is a list and threshold is a float
    """Function returning the corrected Input(list of n neurons)"""
    
    r=[x-threshold for x in Input]  # for item in the list, substract thresold and creat a new list
    r1=[]                           # new list to append corrected value
    for item in r:                   
        if item <= 0:               #if item in list < 0, append 0 to the new list 
            r1.append(0)
        else:                       # if not, append the value Input-threshold
            r1.append(item)
    return(r1)                      #return the last list of corrected inputs

# ici on ferais la soustraction a la somme totale des inputs et pas a chaque synapses(ok?)

# Neurons dynamics: Iteration and parameters update

In [9]:
def UpdateActivity(m,dm): # for n neurons, we add dm of neuron i to the activity m of neuron i
    """Return the sum of the activity dm with the activity m"""
    
    UptoDateActivity=[sum(x) for x in zip(m, dm)]  # permet de faire les sommes respectives de la liste m(activité 
    return UptoDateActivity                        # precedente)
                                                   # et de la variation dm puis de réatribuer cette valeur a m

# Calculation of m(t-delta)

In [10]:
# function to get the index "-delay/dt" in the list representing m(t-Delta) 
m = 0
def mdelta(synapse_value,delay):
    if t < delay:                # if the delay is the time is too small, we take m(t-delta)=0
        m = 0
    elif t >= delay:             # if the time is above or equal to the delay, we can find the m(t-delta)
        m = synapse_value[int(-delay/dt)]   # ???
    return m

In [ ]:
# function to get the index "-delay/dt" in the list representing m(t-Delta) 
m = 0
def mdelta(synapse_value,delay):
    if t < delay:                # if the delay is the time is too small, we take m(t-delta)=0
        delta=bunch[0]
    elif t >= delay:             # if the time is above or equal to the delay, we can find the m(t-delta)
        delta = synapse_value[int(-delay/dt)]   # ???
    return m

In [11]:
def ActivityDelta(bunch,container): # permet de faire la difference entre l'etat n et n-1(=t-delai) dans un array
        """Function giving the difference between the state n-1 and n, to calculate m(t-deltat)"""
        
        if it <= 1 :          # et redonne une liste des differences de chaque neurones dans un array(car en f(t));
            delta=bunch[0]    # la fonction depend de it, l'iteration, et doit etre inclu dans une boucle
        elif it > 1:          # dans laquelle it est incrementé de +1 
            delta=[a_i - b_i for a_i, b_i in zip(bunch[it-1], bunch[it-2])]

        container.append(delta)
        return container            #container must be a list (M1,2,3,4,5,6)

In [12]:
#White Gaussian Noise
def WGnoise():
    n = numpy.random.normal(0,1)
    return n

# Calculation of I and dm updated

In [13]:
#Update of inputs
                                                                                 
def popActuI(population_I,Weight,connectivity_j,container):  
    """Depending on weight,connectivity and the delta activity function(m(t-deltat)),
    returns an updated input for a unique neuron among the chosen population"""
    
    UpdateI = []
    y=0
    while y <= n:
        InputUpdate = [x * c * Weight+ WGnoise() for x, c in zip(connectivity_j[y], container)]
        UpdateI.append(InputUpdate)
        population_I.append(sum(InputUpdate))    # We are taking the sum such that each value is the sum for
        y += 1                                   # of all the inputs of one neuron. Ii(cortex) with i => neuron i

    return population_I

In [14]:
def popActuIGPI():
    GPII=[e - i for e, i in zip(Gpi_Ie, Gpi_Ii)]
    return GPII

In [15]:
#Update of activities

def popActuDM(PopulationM,Input,tau):
    """Depending on population activity, dt, the corrected input, the threshold and tau, 
    returns a small perturbation of the activity dm between the populations selected"""
    
    PopulationActDM =[(-m + i) * dt / tau for m, i in zip(populationM, Input)]
    return PopulationActDM
    
    # l'input de Ic' est celui calculé dans la cellule du dessus=une liste de somme pour une population precise.
    # Ic va renvoyer une liste. PopulationM est une liste.    

#   ======================= Simulation Core =======================

In [16]:
it= 0   #Number of iteration (with step dt) : a simple iteration counter

time_value=[0] #we put in a list every time to retrieve them in a gradual order

# Value of m(t-delta_t) calculated after each iteration ; for all population
M1=[] #CtxTha_m(t-CtxTha_D)=0   # R1 est l'array qui contiendra toute les listes de 
M2=[] #StrCtx_m(t-StrCtx_D)=0   # difference ctxtha (!!!!!le mettre dans la boucle?!!!!!)
M3=[] #StnCtx_m(t-StnCtx_D)=0   
M4=[] #GpiStn_m(t-GpiStn_D)=0   
M5=[] #GpiStr_m(t-GpiStr_D)=0   
M6=[] #ThaGpi_m(t-ThaGpi_D)=0   
# transformation en liste qui contiendra toute les valeurs dont le delai a été soustrait
#servira a faire les operations par la suite: 1ere valeurs = neurones 1 , 2 v=n2, etc

In [17]:
while t < duration:    # when the time is strictly under 1 seconde
    
    t = t+dt           #for each iteration, add (the step) dt to the time
    time_value.append(t)     #command to add every time used in a list
    it += 1            #for each iteration, add 1 to the iteration counter
    
# Update of m for n activities and n dm for each population=> on obtiens une liste dont chaque nombre
# represente l'activité d'un neurone dans la population concernée
    
    CtxTha_m = UpdateActivity(CtxTha_m,CtxTha_dm)    # m est une liste et dm doit etre une liste!
    StnCtx_m = UpdateActivity(StnCtx_m,StnCtx_dm)    # pour pouvoir en faire la somme via la fonction
    StrCtx_m = UpdateActivity(StrCtx_m,StrCtx_dm)    # UpdateActivity
    GpiStr_m = UpdateActivity(GpiStr_m,GpiStr_dm)
    GpiStn_m = UpdateActivity(GpiStn_m,GpiStn_dm)
    ThaGpi_m = UpdateActivity(ThaGpi_m,ThaGpi_dm)
    
    
    
# on creer les listes de listes d'activité, pour chaque population
                                                    
    CtxTha_mbunch = []        # on creer les listes qui vont contenir les listes d'activités en fonction du
    StnCtx_mbunch = []        # temps/iteration
    StrCtx_mbunch = [] 
    GpiStr_mbunch = [] 
    GpiStn_mbunch = []   # !!!!! on ajoute 0 initialement pour que sa corresponde a it=+1 pour les calculs????
    ThaGpi_mbunch = [] 
    
    
    CtxTha_mbunch.append(CtxTha_m)   # une liste qui repertorie les activités dans le temps
    StnCtx_mbunch.append(StnCtx_m)   # de chaque neurones(en liste); 
    StrCtx_mbunch.append(StrCtx_m)  
    GpiStr_mbunch.append(GpiStr_m)
    GpiStn_mbunch.append(GpiStn_m)
    ThaGpi_mbunch.append(ThaGpi_m)
    
    
    #CtxTha_mbunchA = numpy.array(CtxTha_mbunch)
    #StnCtx_mbunchA = numpy.array(StnCtx_mbunch)
    #StrCtx_mbunchA = numpy.array(StrCtx_mbunch)
    #GpiStr_mbunchA = numpy.array(GpiStr_mbunch)
    #GpiStn_mbunchA = numpy.array(GpiStn_mbunch)
    #ThaGpi_mbunchA = numpy.array(ThaGpi_mbunch)     
    
    
    numpy.reshape(CtxTha_mbunch,(it-1,it))   # on creer des array avec n collones et it(=iteration counter) lignes
    numpy.reshape(StnCtx_mbunch,(it-1,it))   # qui vont nous servir ensuite a localiser un neurone precis
    numpy.reshape(StrCtx_mbunch,(it-1,it))   # parmis les reseaux
    numpy.reshape(GpiStr_mbunch,(it-1,it))
    numpy.reshape(GpiStn_mbunch,(it-1,it))
    numpy.reshape(ThaGpi_mbunch,(it-1,it))     
    
    
    
    #====> fonction pour alleger le code:
  
    #def Addnshape(who,add):
    #    who.append(add)
    #    numpy.reshape(who,(it,n))
    # return who

#je laisse en tant que liste de tout mes neurones et je sais que 
#la collone 0 de chaque ligne(=iteration) = activité de mon neurones 1 et ligne = a quel temps.





################################################ WORKING LOOP ###############################################


# Inputs:

    Cortex_I   = popActuI(Cortex_I,1,CtxTha_J,M1) # adapter les connectivity and container(t-deltat)
    Striatum_I = popActuI(Striatum_I,1,StrCtx_J,M2)  
    Stn_I      = popActuI(Stn_I,1,StnCtx_J,M3)
    Gpi_Ie     = popActuI(Gpi_Ie,1,GpiStn_J,M4) 
    Gpi_Ii     = popActuI(Gpi_Ii,1,GpiStr_J,M5) 
    Gpi_I      = popActuIGPI()
    Thalamus_I = popActuI(Thalamus_I,1,ThaGpi_J,M6) 
    
    
# Activities:    
    
    CtxTha_dm = popActuDM(CtxTha_m,Thalamus_I,CtxTha_tau)
    StrCtx_dm = popActuDM(StrCtx_m,Cortex_I,StrCtx_tau)
    StnCtx_dm = popActuDM(StnCtx_m,Cortex_I,StnCtx_tau)
    GpiStr_dm = popActuDM(GpiStr_m,Striatum_I,GpiStr_tau)
    GpiStn_dm = popActuDM(GpiStn_m,Stn_I,GpiStn_tau)
    ThaGpi_dm = popActuDM(ThaGpi_m,Gpi_I,ThaGpi_tau)
    

IndexError: index 2 is out of bounds for axis 0 with size 2

# Plotting results

In [ ]:
#plt.plot(time_value,CtxTha_value,label='Thalamus')
#plt.plot(time_value,StrCtx_value,label='Cortex')
#plt.plot(time_value,StnCtx_value,label='Cortex')
#plt.plot(time_value,GpiStn_value,label='STN')
#plt.plot(time_value,GpiStr_value,label='Striatum')
#plt.plot(time_value,ThaGpi_value,'k',label='Gpi')

#plt.title('Activity over time')
#plt.ylabel('Activity')
#plt.xlabel('time')
#plt.grid()

#plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

#plt.show()